In [2]:
import json
from data.preprocess import DataProcessor
import os
from data.custom_dataset import CustomDataset
from data.utils import get_dataset, get_tokenizer, get_dataloader
from model.model import Model
import torch
import pandas as pd
import numpy as np
import torch.optim as optim
import ast
import os
import itertools
import torch.utils.checkpoint
from torch.cuda.amp import GradScaler, autocast
from training.trainer import Trainer
import argparse



/vol/bitbucket/mh1022/dl_cw_pyenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'ipdbb'

In [3]:
config = {
#    "run_name": "Run_test_TLWAN"
    "run_name": "test"
    ,"project_path": '/vol/bitbucket/mh1022/temporal-modelling-icd'
    ,"base_checkpoint": os.path.join("", "RoBERTa-base-PM-M3-Voc-hf")
    ,"num_attention_heads": 1
    ,"num_layers": 1
    ,"lr": 5e-5
    ,"max_chunks": 4
    ,"grad_accumulation": 4
    ,"use_positional_embeddings": True
    ,"use_reverse_positional_embeddings": True
    ,"priority_mode": "Last"
    ,"priority_idxs": [1]
    ,"use_document_embeddings": True
    ,"use_reverse_document_embeddings": True
    ,"use_category_embeddings": True
    ,"num_labels": 50
    ,"use_all_tokens": True
    ,"final_aggregation": "cls"
    ,"only_discharge_summary": False
    ,"patience_threshold": 3
    ,"max_epochs": 20
    ,"save_model": True
    ,"load_from_checkpoint": False
    ,"checkpoint_name": "Run_all_notes_last_second_transf"
}

In [4]:
with open(os.path.join("", f"results/config_{config['run_name']}.json"), "w") as f:
    json.dump(config, f)

# process and aggregate raw data
dp = DataProcessor(dataset_path="dataset", config=config)
notes_agg_df = dp.aggregate_data()
categories_mapping = dp.get_categories_mapping()


/vol/bitbucket/mh1022/temporal-modelling-icd/data/preprocess.py:16: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  self.notes_df = pd.read_csv(os.path.join(dataset_path, "NOTEEVENTS.csv"))


{'Radiology': 0, 'Nursing/other': 1, 'ECG': 2, 'Nursing': 3, 'Discharge summary': 4, 'Physician ': 5, 'Echo': 6, 'Nutrition': 7, 'Respiratory ': 8, 'General': 9}


In [15]:
notes_agg_df.loc[19,:].CATEGORY

['Radiology',
 'ECG',
 'Nursing/other',
 'Nursing/other',
 'Nursing/other',
 'Nursing/other',
 'Nursing/other',
 'Nursing/other',
 'Nursing/other',
 'Nursing/other',
 'Nursing/other',
 'Nursing/other',
 'Nursing/other',
 'Radiology',
 'Echo',
 'Nursing/other',
 'Nursing/other',
 'Nursing/other',
 'Radiology',
 'Radiology',
 'Discharge summary',
 'Discharge summary']

In [21]:
text_joined = " "
for category, text in zip(notes_agg_df.loc[19,:].CATEGORY, notes_agg_df.loc[19,:].TEXT):
    text_joined += f"\n#####################This is a {category} note################\n"
    text_joined += text

In [22]:
with open("text.txt", 'w') as f:
    f.writelines(text_joined)